In [1]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [2]:
import tensorflow.keras as K

In [3]:
def preprocess_data(X,Y):
    X_p = K.applications.resnet50.preprocess_input(X)
    Y_p = K.utils.to_categorical(Y,10)
    return X_p,Y_p

In [4]:
(x_train,y_train),(x_test,y_test) = K.datasets.cifar10.load_data()
print(x_train.shape,y_train.shape)
x_train,y_train = preprocess_data(x_train,y_train)
x_test,y_test= preprocess_data(x_test,y_test)
print(x_train.shape,y_train.shape)

(50000, 32, 32, 3) (50000, 1)
(50000, 32, 32, 3) (50000, 10)


In [5]:
input_t = K.Input(shape=(32,32,3))
res_model = K.applications.ResNet50(weights='imagenet',include_top=False,input_tensor=input_t)

In [10]:
for layer in res_model.layers[:143]:
    layer.trainable = False

In [11]:
for i,layer in enumerate(res_model.layers):
    print(i,layer.name,"-",layer.trainable)

0 input_1 - False
1 conv1_pad - False
2 conv1_conv - False
3 conv1_bn - False
4 conv1_relu - False
5 pool1_pad - False
6 pool1_pool - False
7 conv2_block1_1_conv - False
8 conv2_block1_1_bn - False
9 conv2_block1_1_relu - False
10 conv2_block1_2_conv - False
11 conv2_block1_2_bn - False
12 conv2_block1_2_relu - False
13 conv2_block1_0_conv - False
14 conv2_block1_3_conv - False
15 conv2_block1_0_bn - False
16 conv2_block1_3_bn - False
17 conv2_block1_add - False
18 conv2_block1_out - False
19 conv2_block2_1_conv - False
20 conv2_block2_1_bn - False
21 conv2_block2_1_relu - False
22 conv2_block2_2_conv - False
23 conv2_block2_2_bn - False
24 conv2_block2_2_relu - False
25 conv2_block2_3_conv - False
26 conv2_block2_3_bn - False
27 conv2_block2_add - False
28 conv2_block2_out - False
29 conv2_block3_1_conv - False
30 conv2_block3_1_bn - False
31 conv2_block3_1_relu - False
32 conv2_block3_2_conv - False
33 conv2_block3_2_bn - False
34 conv2_block3_2_relu - False
35 conv2_block3_3_conv - 

In [12]:
img_size_resnet=(224,224)
model = K.models.Sequential()
model.add(K.layers.Lambda(lambda image: tf.image.resize(image,img_size_resnet)))
model.add(res_model)
model.add(K.layers.Flatten())
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(256,activation='relu'))
model.add(K.layers.Dropout(0.5))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(128,activation='relu'))
model.add(K.layers.Dropout(0.5))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(64,activation='relu'))
model.add(K.layers.Dropout(0.5))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(10,activation='softmax'))

In [16]:
checkpoint = K.callbacks.ModelCheckpoint(filepath='cifar10.h5',monitor="val_acc",mode="max",save_best_only=True)

In [ ]:
model.compile(loss="categorical_crossentropy",optimizer=K.optimizers.Adam(lr=2e-5),metrics=['accuracy'])
history = model.fit(x_train,y_train,batch_size=32,epochs=10,verbose=1,validation_data=(x_test,y_test),callbacks=[checkpoint])
model.summary()
model.save("cifar10.h5")

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
25248/50000 [==============>...............] - ETA: 10:56:55 - loss: 2.2975 - accuracy: 0.2353